## 1) Uploading data

In [1]:
import sklearn
import numpy as np
import random
import re
from numpy.linalg import norm

In [2]:
COLAB = True

In [3]:
if COLAB:
    from google.colab import drive
    from os.path import exists

    amazon_train_file = 'train.ft.txt.bz2'

    if exists(amazon_train_file):
        train_file = amazon_train_file
    else:
        drive.mount('/content/drive/')

        # For Hadas' drive
        #my_dir = 'drive/MyDrive/Y-data/Intuit-K-anonimity/'

        # For Lior's drive
        my_dir = 'drive/MyDrive/Y-data/Y-DATA_PROJECT/'

        train_file = my_dir + '/train.ft.txt.bz2'
else:
    train_file = '../data/' + 'train.ft.txt.bz2'

Mounted at /content/drive/


## 2) Collecting the data

In [4]:
# Credit https://www.kaggle.com/code/anshulrai/cudnnlstm-implementation-93-7-accuracy

import bz2

# Readling the file to list of comments
train_file = bz2.BZ2File(train_file)
train_file_lines = train_file.readlines()

# Converting from raw binary strings to strings that can be parsed
train_file_lines = [x.decode('utf-8') for x in train_file_lines]

# Extracting the labels and sentences
#train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines] # And converting to lower case

del(train_file_lines)  # Free RAM


## 3) Choosing short sentence

In [5]:
len(train_sentences)

3600000

In [6]:
# choose only sentenses with less than 20 words
short_train_sentences_total = [x for x in train_sentences if len(x.split(' ')) < 25]

In [7]:
# how many sentences we got;
len(short_train_sentences_total)

190275

In [8]:
size = len(short_train_sentences_total)
size

190275

In [15]:
short_train_sentences = random.sample(short_train_sentences_total, 1500)  # choosing only 1500

In [ ]:
# take only first 1000 (is it suffled?NO)
ts = short_train_sentences[:1000]

## 4) Take a look at a few of random (short) sentences:

In [16]:
print(short_train_sentences[0])
print(len(short_train_sentences[0].split(' ')))

worked great...for a couple weeks: then it started giving wildly inaccurate readings and shutting off randomly during use. don't buy it!
21


In [17]:
print(short_train_sentences[10])
print(len(short_train_sentences[10].split(' ')))

excellent: very nice book. our daughters love it and spend a lot of time with it. makes a nice present to little readers.
23


In [18]:
print(short_train_sentences[110])
print(len(short_train_sentences[110].split(' ')))

horrible: this is the worst movie i have ever seen. a complete waste of time and money. so corny, hokey and stupid.
22


## 5) Very initial Pre-Processing:


In [19]:
def word_extraction(sentence):    
  ''' gets a sentence and returns a 
  list of words = BoW  '''
  ignore = []    # for now we will not igone any words
  words = re.sub("[^\w]", " ",  sentence).split()    
  cleaned_text = [w.lower() for w in words if w not in ignore] # i'm not sure we need to lower - all are lower already
  return cleaned_text

In [21]:
# original
print(short_train_sentences[0])
print(len(short_train_sentences[0].split(' ')))
# proccesed
print(word_extraction(short_train_sentences[0]))
print(len(word_extraction(short_train_sentences[0])))

# for some reason "Don't" was splited to "don" and "t" :(:(:()))

worked great...for a couple weeks: then it started giving wildly inaccurate readings and shutting off randomly during use. don't buy it!
21
['worked', 'great', 'for', 'a', 'couple', 'weeks', 'then', 'it', 'started', 'giving', 'wildly', 'inaccurate', 'readings', 'and', 'shutting', 'off', 'randomly', 'during', 'use', 'don', 't', 'buy', 'it']
23


In [22]:
# original
print(short_train_sentences[10])
print(len(short_train_sentences[10].split(' ')))
# proccesed
print(word_extraction(short_train_sentences[10]))
print(len(word_extraction(short_train_sentences[10])))

excellent: very nice book. our daughters love it and spend a lot of time with it. makes a nice present to little readers.
23
['excellent', 'very', 'nice', 'book', 'our', 'daughters', 'love', 'it', 'and', 'spend', 'a', 'lot', 'of', 'time', 'with', 'it', 'makes', 'a', 'nice', 'present', 'to', 'little', 'readers']
23


In [23]:
# original
print(short_train_sentences[110])
print(len(short_train_sentences[110].split(' ')))
# proccesed
print(word_extraction(short_train_sentences[110]))
print(len(word_extraction(short_train_sentences[110])))

horrible: this is the worst movie i have ever seen. a complete waste of time and money. so corny, hokey and stupid.
22
['horrible', 'this', 'is', 'the', 'worst', 'movie', 'i', 'have', 'ever', 'seen', 'a', 'complete', 'waste', 'of', 'time', 'and', 'money', 'so', 'corny', 'hokey', 'and', 'stupid']
22


##  6) Using Sentence embedding to find 2 similar sentences

In [24]:
!pip install sentence_transformers -q  # https://www.sbert.net/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.1 MB/s eta 0:00:00


In [25]:
from sentence_transformers import SentenceTransformer

In [26]:
first_sentence = short_train_sentences[0] # choosing one sentence

### for first step we chose the following model

In [27]:
# This is a sentence-transformers model: https://huggingface.co/sentence-transformers/all-MiniLM-L12-v1
# It maps sentences & paragraphs to a 384 dimensional dense vector space 
# and can be used for tasks like clustering or semantic search.
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1') # creating a model

#### 6.1) Embeding only one sentence as an example

In [47]:
embeddings_one = model.encode(first_sentence)

In [48]:
max(embeddings_one) # do we need to normalize?

0.14359021

In [49]:
min(embeddings_one) # do we need to normalize?

-0.161408

In [50]:
print(len(embeddings_one)) # indeed mapped to a 384 dim vector

384


#### 6.2) Embedding the chosen short sentences
we would like to find couples of most similar sentences to perform K-Anonimization on:)

In [51]:
## model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1') # creating a model
## takes alot of time on this number of sentences..
%%time
embeddings_all = model.encode(short_train_sentences)  # embedding on all 190275 sentences is too long. working on 1500 sentences

CPU times: user 1min 8s, sys: 184 ms, total: 1min 9s
Wall time: 1min 23s


In [52]:
np.size(embeddings_all[0])


384

#### 6.3) Distance Matrix:
Some of the popular similarity measures are –

Euclidean Distance.
Manhattan Distance.
Jaccard Similarity.
Minkowski Distance.
Cosine Similarity.

we chose to start with Cosine-sim

In [33]:
def calc_cosine_sim(sent_A, sent_B):
  # compute cosine similarity
  cosine = np.dot(sent_A, sent_B)/(norm(sent_A)*norm(sent_B))
  return cosine

In [53]:
ind=0
one_sent = short_train_sentences[ind]
one_sent

"worked great...for a couple weeks: then it started giving wildly inaccurate readings and shutting off randomly during use. don't buy it!"

In [54]:
calc_cosine_sim(embeddings_all[0], embeddings_all[0])  # sanity check

1.0

In [55]:
sim = [(calc_cosine_sim(embeddings_one, e), idx) for idx, e in enumerate(embeddings_all)]
#sim = [(calc_cosine_sim(embeddings_one, e), ind, idx) for idx, e in enumerate(embeddings_all)] # (score, sent_ind, sent_ind_2)

In [56]:
sim.sort(reverse=True)

In [57]:
sim[1]  # similarity is not very good.. let's find a differnet per (not 0,180)

(0.5945619, 493)

In [68]:
sim_list = []
list(sim_list)
print(type(sim_list))

<class 'list'>


In [69]:
np.shape(embeddings_all)

(1500, 384)

In [70]:
for ind in range(0,len(short_train_sentences)-1):
    embeddings_one = model.encode(short_train_sentences[ind]) # choosing one sent each loop
    sim = [(calc_cosine_sim(embeddings_one, e), ind, idx) for idx, e in enumerate(embeddings_all)] # (score, sent_ind, sent_ind_2)
    sim.sort(reverse=True)
    #print(sim[1]) # print most similar score and index which is not the same sentences
    sim_list.append(sim[1])

In [71]:
len(short_train_sentences)

1500

In [72]:
sim_list.sort(reverse=True)
sim_list[0:10]

[(0.8503182, 535, 1370),
 (0.8503181, 1370, 535),
 (0.82217485, 131, 74),
 (0.8221747, 74, 131),
 (0.813788, 1247, 74),
 (0.80414045, 872, 398),
 (0.8041403, 398, 872),
 (0.8027349, 10, 203),
 (0.80273473, 203, 10),
 (0.7969144, 824, 818)]

In [75]:
# print most similar sentences:
print(short_train_sentences[535])
print(short_train_sentences[1370])

awful: this movie was awful. the storyline, the characters, and the music....all awful. it was hard to watch all the way through.
awful: this movie was just absolutely awful; the story line was ok but needed much improvement, the cast were cheesy; regret buying this one!


In [73]:
sim_list = []
list(sim_list)
print(type(sim_list))

<class 'list'>


In [74]:
# same but with broadcasting with the help of CHATGPT
#### NOT WORKING YET ####
short_train_embeddings = model.encode(short_train_sentences)
cosine_sims = calc_cosine_sim(short_train_embeddings[:, np.newaxis, :], embeddings_all)
max_sims = np.max(cosine_sims, axis=2)
max_idxs = np.argmax(cosine_sims, axis=2)
for i in range(len(short_train_sentences)-1):
    sim_list.append((max_sims[i, max_idxs[i]], i, max_idxs[i]))
###

ValueError: ignored

In [ ]:
sim_list.sort(reverse=True)
sim_list[0:10]

## Algorithm for k=2 anonymity:
1) take a sentence and the next most similar sentence

2) look at the BoW, and if need sort them.

3) remove word that appear in both to better view what is left

4) embbed using Glove.

5) find 5 most similar words to each word in the sentence

6) if common words for both of the sentences - switch to it
.
else - use DBScan to switch word with Centroid or else.



In [76]:
first_sentence

#  most similar sentences:
first_sentence = short_train_sentences[535]
closest_sentence = short_train_sentences[1370]

In [77]:
# Taken from: https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/
import re
first_sentence_bow = re.sub(r"[^a-zA-Z0-9]", " ", first_sentence.lower()).split() # lower is probably redundant 
closest_sentence_bow = re.sub(r"[^a-zA-Z0-9]", " ", closest_sentence.lower()).split() # lower is probably redundant 




In [78]:
print("len of first sentence",len(first_sentence_bow))
print("len of similar sentence",len(closest_sentence_bow))
print("first sent BoW:",first_sentence_bow)
print("closest sent BoW:",closest_sentence_bow)


len of first sentence 23
len of similar sentence 24
first sent BoW: ['awful', 'this', 'movie', 'was', 'awful', 'the', 'storyline', 'the', 'characters', 'and', 'the', 'music', 'all', 'awful', 'it', 'was', 'hard', 'to', 'watch', 'all', 'the', 'way', 'through']
closest sent BoW: ['awful', 'this', 'movie', 'was', 'just', 'absolutely', 'awful', 'the', 'story', 'line', 'was', 'ok', 'but', 'needed', 'much', 'improvement', 'the', 'cast', 'were', 'cheesy', 'regret', 'buying', 'this', 'one']


In [79]:
# Sort the BoW:
print("first sent BoW:",sorted(first_sentence_bow))
print("closest sent BoW:",sorted(closest_sentence_bow))

first sent BoW: ['all', 'all', 'and', 'awful', 'awful', 'awful', 'characters', 'hard', 'it', 'movie', 'music', 'storyline', 'the', 'the', 'the', 'the', 'this', 'through', 'to', 'was', 'was', 'watch', 'way']
closest sent BoW: ['absolutely', 'awful', 'awful', 'but', 'buying', 'cast', 'cheesy', 'improvement', 'just', 'line', 'movie', 'much', 'needed', 'ok', 'one', 'regret', 'story', 'the', 'the', 'this', 'this', 'was', 'was', 'were']


In [80]:
# Let's find the common words and remove them, so we can consentrate on the rest:
first_common = [word for word in first_sentence_bow if word in closest_sentence_bow]
first_common

['awful',
 'this',
 'movie',
 'was',
 'awful',
 'the',
 'the',
 'the',
 'awful',
 'was',
 'the']

In [81]:
second_common = [word for word in closest_sentence_bow if word in first_sentence_bow]
second_common

['awful', 'this', 'movie', 'was', 'awful', 'the', 'was', 'the', 'this']

In [82]:
list_common = second_common  # words to remove from both sentenses

In [83]:
# remove from both sentenses
first_filtered = [word for word in first_sentence_bow if word not in list_common ]
seond_filtered = [word for word in closest_sentence_bow if word not in list_common ]


In [84]:
sorted(first_filtered)
first_filtered

['storyline',
 'characters',
 'and',
 'music',
 'all',
 'it',
 'hard',
 'to',
 'watch',
 'all',
 'way',
 'through']

In [87]:
sorted(seond_filtered)
seond_filtered

['just',
 'absolutely',
 'story',
 'line',
 'ok',
 'but',
 'needed',
 'much',
 'improvement',
 'cast',
 'were',
 'cheesy',
 'regret',
 'buying',
 'one']

In [ ]:
# Word Embedding and find similar

In [88]:
import gensim.downloader as api
glove_model = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
embedded_first_filtered = glove_model[first_filtered]  # embedding to vector space each word of first sentense
embedded_second_filtered = glove_model[seond_filtered]  # embedding to vector space each word of second sentense

(12, 25)

### Using Glove to try and find similar words in both BoWs:

In [99]:
glove_model.similar_by_word('regret')

[('part', 0.9296597838401794),
 ('movie', 0.9051793813705444),
 ('thing', 0.9006426930427551),
 ('music', 0.9006094336509705),
 ('secret', 0.8985744118690491),
 ('song', 0.8981215357780457),
 ('one', 0.8963617086410522),
 ('life', 0.8931435346603394),
 ('read', 0.8918521404266357),
 ('this', 0.891066312789917)]

# Seq2Seq example

In [90]:
!pip install transformers -q

In [91]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "snrspeaks/t5-one-line-summary"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [92]:
def sum_text(doc_list):
    print('Inputs:')
    for d in doc_list:
        print(d)
    abstract = '. '.join(doc_list)
    #print('\nAbstract:\n', abstract)
    input_ids = tokenizer.encode("summarize: " + abstract, return_tensors="pt", add_special_tokens=True)
    generated_ids = model.generate(input_ids=input_ids,num_beams=5,max_length=50,repetition_penalty=2.5,length_penalty=1,early_stopping=True,num_return_sequences=3)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    print('\nOutput:\n', preds)
    return preds

In [97]:
doc1 = short_train_sentences[535]
doc2 = short_train_sentences[1370]
sum_text([doc1, doc2]);

Inputs:
awful: this movie was awful. the storyline, the characters, and the music....all awful. it was hard to watch all the way through.
awful: this movie was just absolutely awful; the story line was ok but needed much improvement, the cast were cheesy; regret buying this one!

Output:
 ['A Very Bad Movie', 'Awfull', 'Anomalous Things in X-Rays']


## My Suggested Output:
### "awful moview", "bad story-line", "regret buying"
## My BoW Suggestion Output:
### "awful", "movie", "Regert","all", "buy"

# options how to continue:
1) choose another 2 close sentences

2) change dist matrix (Jaccard/else)

3) perform more pre-process prior to sentence embedding? might be redundant

4) use "word embedding" model to try and find the distance between the words in both sentences

5) else?

In [ ]:
###

In [100]:
### 1) choose another 2 close sentences (131, 74)


In [102]:
# print most similar sentences:
print(short_train_sentences[131])
print(short_train_sentences[74])

great seller: the product was shipped very quickly and was exactly as it was described in the ad great condition thank you
very satisfied: i received a great product in great condition just as it was described. the shipping was fairly fast as well!


In [ ]:
#  most similar sentences:
first_sentence = short_train_sentences[131]
closest_sentence = short_train_sentences[74]

In [104]:
# Taken from: https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/
import re
first_sentence_bow = re.sub(r"[^a-zA-Z0-9]", " ", first_sentence.lower()).split() # lower is probably redundant 
closest_sentence_bow = re.sub(r"[^a-zA-Z0-9]", " ", closest_sentence.lower()).split() # lower is probably redundant 

In [105]:
sorted(first_filtered)
first_filtered

['storyline',
 'characters',
 'and',
 'music',
 'all',
 'it',
 'hard',
 'to',
 'watch',
 'all',
 'way',
 'through']

In [106]:
sorted(seond_filtered)
seond_filtered

['just',
 'absolutely',
 'story',
 'line',
 'ok',
 'but',
 'needed',
 'much',
 'improvement',
 'cast',
 'were',
 'cheesy',
 'regret',
 'buying',
 'one']

In [107]:
doc1 = short_train_sentences[131]
doc2 = short_train_sentences[74]
sum_text([doc1, doc2]);

Inputs:
great seller: the product was shipped very quickly and was exactly as it was described in the ad great condition thank you
very satisfied: i received a great product in great condition just as it was described. the shipping was fairly fast as well!

Output:
 ['Very satisfied with the product', 'Very satisfied with the delivery of the product', 'Very satisfied: a very good seller']


## My Suggested Output:
### "great product", "very satisfied", "fast shipping"
## My BoW Suggestion Output:
### "great", "product", "fast","delivery", "satisfied"